In [1]:
import pandas as pd


full_data = pd.read_excel(r"D:\Downloads\DSTC vòng 3\raw data\HNXINDEX.xlsx")
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277930 entries, 0 to 277929
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  277930 non-null  int64  
 1   ticker      277930 non-null  object 
 2   timestamp   277930 non-null  object 
 3   open        277930 non-null  float64
 4   high        277930 non-null  float64
 5   low         277930 non-null  float64
 6   close       277930 non-null  float64
 7   volume      277930 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 17.0+ MB


In [2]:
from FiinQuantX import FiinSession
import pandas as pd
username = 'DSTC_28@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(username=username, password=password).login()
fi = client.FiinIndicator()

In [4]:
counts = full_data.groupby("ticker").size()
counts.describe()

count    304.000000
mean     914.243421
std       51.238071
min       63.000000
25%      919.000000
50%      919.000000
75%      919.000000
max      919.000000
dtype: float64

In [5]:
list_keep = counts[counts == 919].index
list_keep.size

290

In [6]:
df = full_data[full_data['ticker'].isin(list_keep)].reset_index()
df = df.drop("Unnamed: 0", axis = 1)
df = df.drop("index", axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266510 entries, 0 to 266509
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ticker     266510 non-null  object 
 1   timestamp  266510 non-null  object 
 2   open       266510 non-null  float64
 3   high       266510 non-null  float64
 4   low        266510 non-null  float64
 5   close      266510 non-null  float64
 6   volume     266510 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 14.2+ MB


In [7]:
df.head()

,ticker,timestamp,open,high,low,close,volume
0,VTZ,2022-01-04 00:00,9239.943496,9472.883248,9162.296912,9472.883248,181800
1,VTZ,2022-01-05 00:00,9550.529832,9938.762752,9550.529832,9628.176416,88300
2,VTZ,2022-01-06 00:00,9628.176416,9628.176416,9317.590080,9317.590080,92400
3,VTZ,2022-01-07 00:00,9472.883248,9472.883248,9239.943496,9472.883248,120200
4,VTZ,2022-01-10 00:00,9395.236664,9938.762752,9395.236664,9395.236664,209100


In [8]:
df.tail()

,ticker,timestamp,open,high,low,close,volume
266505,X20,2025-09-08 00:00,13300.0,13300.0,13300.0,13300.0,1000
266506,X20,2025-09-09 00:00,13200.0,13200.0,13200.0,13200.0,200
266507,X20,2025-09-10 00:00,13500.0,13600.0,13500.0,13600.0,10100
266508,X20,2025-09-11 00:00,13500.0,13600.0,13500.0,13600.0,33100
266509,X20,2025-09-12 00:00,13500.0,13600.0,13500.0,13600.0,7100


In [12]:
zero_counts_per_ticker = df["volume"].eq(0).groupby(df["ticker"]).sum()

# 2) Lấy 16 ticker có số lần volume=0 ít nhất
min_nonzero = zero_counts_per_ticker[zero_counts_per_ticker > 0]
tickers_min_10 = min_nonzero.nsmallest(10).index

df_zero_min10 = df[(df["ticker"].isin(tickers_min_10)) & (df["volume"] == 0)]

# 4) Lấy danh sách timestamp
timestamps_min10 = df_zero_min10["timestamp"].unique()


print("Timestamp mà 10 ticker ít volume=0 nhất có volume=0:")
print(timestamps_min10)

Timestamp mà 10 ticker ít volume=0 nhất có volume=0:
['2022-01-12 00:00' '2022-01-28 00:00' '2022-02-14 00:00'
 '2022-03-29 00:00' '2022-03-31 00:00' '2023-02-27 00:00'
 '2023-03-31 00:00' '2022-06-17 00:00' '2024-09-11 00:00'
 '2024-09-12 00:00' '2024-11-25 00:00' '2025-06-25 00:00'
 '2025-06-30 00:00' '2025-07-03 00:00' '2023-03-20 00:00'
 '2025-06-05 00:00' '2023-12-14 00:00' '2024-10-11 00:00'
 '2024-11-15 00:00' '2025-04-21 00:00' '2025-05-08 00:00'
 '2025-06-12 00:00' '2025-06-17 00:00' '2025-09-10 00:00'
 '2024-04-17 00:00' '2024-10-04 00:00' '2025-01-20 00:00'
 '2024-01-22 00:00' '2024-01-25 00:00' '2024-01-26 00:00'
 '2024-08-15 00:00']


In [13]:
df_filtered = df[~df["timestamp"].isin(timestamps_min16)]
df_filtered["timestamp"].nunique()

888

In [14]:
tickers_with_zero = df_filtered.loc[df_filtered['volume'] == 0, 'ticker'].unique()
print(tickers_with_zero.size)

214


In [15]:
df_clean = df_filtered[~df_filtered['ticker'].isin(tickers_with_zero)].copy()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67488 entries, 919 to 262833
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     67488 non-null  object 
 1   timestamp  67488 non-null  object 
 2   open       67488 non-null  float64
 3   high       67488 non-null  float64
 4   low        67488 non-null  float64
 5   close      67488 non-null  float64
 6   volume     67488 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 4.1+ MB


In [16]:
print(df_clean['ticker'].unique())

['AAV' 'API' 'APS' 'BNA' 'BCC' 'BVS' 'C69' 'CAP' 'CEO' 'CMS' 'CSC' 'DHT'
 'DL1' 'DST' 'DXP' 'HBS' 'HLD' 'HOM' 'HUT' 'IDC' 'IDJ' 'IDV' 'DDG' 'IPA'
 'ITQ' 'IVS' 'GKM' 'L14' 'L18' 'LAS' 'LIG' 'MST' 'NAG' 'BAB' 'NBC' 'NDN'
 'NRC' 'NTP' 'NVB' 'EVS' 'PLC' 'PSD' 'PSI' 'KSF' 'DTD' 'PV2' 'PVB' 'PVC'
 'PSW' 'PVG' 'PVI' 'PVS' 'S99' 'SCG' 'SCI' 'SD9' 'NSH' 'SHS' 'TDT' 'THD'
 'THT' 'TTH' 'TIG' 'MBS' 'TNG' 'TVD' 'VC2' 'VC3' 'VC7' 'VCS' 'VGS' 'MBG'
 'VIG' 'VHE' 'VNR' 'VTV']


In [17]:
df_clean = df_clean.reset_index()
df_clean = df_clean.drop("index", axis =1)


In [18]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67488 entries, 0 to 67487
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     67488 non-null  object 
 1   timestamp  67488 non-null  object 
 2   open       67488 non-null  float64
 3   high       67488 non-null  float64
 4   low        67488 non-null  float64
 5   close      67488 non-null  float64
 6   volume     67488 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 3.6+ MB


In [19]:
df_clean.tail(20)

,ticker,timestamp,open,high,low,close,volume
67468,VTV,2025-08-13 00:00,11800.0,11800.0,11600.0,11700.0,46800
67469,VTV,2025-08-14 00:00,11500.0,11800.0,11400.0,11800.0,25800
67470,VTV,2025-08-15 00:00,11700.0,11900.0,11600.0,11900.0,48100
67471,VTV,2025-08-18 00:00,12000.0,12000.0,12000.0,12000.0,1900
67472,VTV,2025-08-19 00:00,11500.0,12000.0,11500.0,12000.0,13700
67473,VTV,2025-08-20 00:00,11600.0,12000.0,11500.0,12000.0,13900
67474,VTV,2025-08-21 00:00,11600.0,12000.0,11600.0,12000.0,3300
67475,VTV,2025-08-22 00:00,11600.0,12000.0,11500.0,12000.0,15300
67476,VTV,2025-08-25 00:00,11600.0,12000.0,11600.0,12000.0,9500
67477,VTV,2025-08-26 00:00,11600.0,12000.0,11500.0,12000.0,49800


In [20]:
import numpy as np

tickers = df_clean['ticker'].unique()
all_dfs = []

    
for t in tickers:
    df_t = df_clean[df_clean['ticker'] == t].copy()

    df_t['ema_50'] = fi.ema(df_t['close'], window=50)
    df_t['ema_200'] = fi.ema(df_t['close'], window=200)
    df_t['macd'] = fi.macd(df_t['close'], window_fast=12, window_slow=26)
    df_t['macd_signal'] = fi.macd_signal(df_t['close'], window_fast=12, window_slow=26, window_sign=9)
    df_t['macd_diff'] = fi.macd_diff(df_t['close'], window_fast=12, window_slow=26, window_sign=9)
    df_t['rsi'] = fi.rsi(df_t['close'], window=14) 
    df_t['bollinger_hband'] = fi.bollinger_hband(df_t['close'], window=20, window_dev=2)
    df_t['bollinger_lband'] = fi.bollinger_lband(df_t['close'], window=20, window_dev=2)
    df_t['mfi'] = fi.mfi(df_t['high'], df_t['low'], df_t['close'], df_t['volume'], window=14) 

    df_t['ema_200'] = np.where(df_t['ema_200'].isna(), df_t['ema_50'], df_t['ema_200'])

    all_dfs.append(df_t)



df_indicators = pd.concat(all_dfs, ignore_index=True)

print(df_indicators.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67488 entries, 0 to 67487
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ticker           67488 non-null  object 
 1   timestamp        67488 non-null  object 
 2   open             67488 non-null  float64
 3   high             67488 non-null  float64
 4   low              67488 non-null  float64
 5   close            67488 non-null  float64
 6   volume           67488 non-null  int64  
 7   ema_50           63764 non-null  float64
 8   ema_200          63764 non-null  float64
 9   macd             65588 non-null  float64
 10  macd_signal      64980 non-null  float64
 11  macd_diff        64980 non-null  float64
 12  rsi              66500 non-null  float64
 13  bollinger_hband  66044 non-null  float64
 14  bollinger_lband  66044 non-null  float64
 15  mfi              66463 non-null  object 
dtypes: float64(12), int64(1), object(3)
memory usage: 8.2+ MB


In [21]:
df_indicators.head(20)

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,macd_signal,macd_diff,rsi,bollinger_hband,bollinger_lband,mfi
0,AAV,2022-01-04 00:00,22802.32,24638.10,22802.32,24058.38,756200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAV,2022-01-05 00:00,24058.38,24541.48,23671.90,23768.52,586500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAV,2022-01-06 00:00,23768.52,24155.00,23575.28,23671.90,710000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAV,2022-01-07 00:00,23671.90,23865.14,23092.18,23092.18,953000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAV,2022-01-10 00:00,23188.80,23961.76,22995.56,23092.18,845700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AAV,2022-01-11 00:00,23188.80,23382.04,22415.84,22705.70,510200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AAV,2022-01-13 00:00,21546.26,21932.74,19807.10,19807.10,735400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AAV,2022-01-14 00:00,19807.10,21739.50,19613.86,21739.50,588300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AAV,2022-01-17 00:00,22125.98,23768.52,20773.30,20773.30,623300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AAV,2022-01-18 00:00,20290.20,21739.50,20290.20,20580.06,337300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_cleaned = df_indicators.dropna(how = 'any').copy().reset_index()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63727 entries, 0 to 63726
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            63727 non-null  int64  
 1   ticker           63727 non-null  object 
 2   timestamp        63727 non-null  object 
 3   open             63727 non-null  float64
 4   high             63727 non-null  float64
 5   low              63727 non-null  float64
 6   close            63727 non-null  float64
 7   volume           63727 non-null  int64  
 8   ema_50           63727 non-null  float64
 9   ema_200          63727 non-null  float64
 10  macd             63727 non-null  float64
 11  macd_signal      63727 non-null  float64
 12  macd_diff        63727 non-null  float64
 13  rsi              63727 non-null  float64
 14  bollinger_hband  63727 non-null  float64
 15  bollinger_lband  63727 non-null  float64
 16  mfi              63727 non-null  object 
dtypes: float64(1

In [23]:
df_cleaned = df_cleaned.drop("index", axis =1)
df_cleaned.head()

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,macd_signal,macd_diff,rsi,bollinger_hband,bollinger_lband,mfi
0,AAV,2022-03-24 00:00,22415.84,22512.46,21836.12,22125.98,236100,22915.445513,22915.445513,-298.539403,-247.711485,-50.827918,43.357123,24208.233901,21193.504099,20.534903
1,AAV,2022-03-25 00:00,22415.84,22512.46,21932.74,22029.36,285300,22880.697062,22880.697062,-303.332986,-258.835785,-44.497201,42.374002,24147.442352,21128.689648,22.420285
2,AAV,2022-03-28 00:00,22029.36,22029.36,21449.64,21642.88,426300,22832.155216,22832.155216,-334.462170,-273.961062,-60.501108,38.603353,24063.272540,21029.281460,23.695297
3,AAV,2022-03-30 00:00,21739.50,21739.50,20193.58,20869.92,478700,22755.204816,22755.204816,-416.700223,-302.508894,-114.191329,32.394605,23930.189747,20853.180253,24.491967
4,AAV,2022-04-01 00:00,20676.68,20676.68,20193.58,20580.06,380000,22669.905019,22669.905019,-499.505761,-341.908268,-157.597494,30.418830,23849.547356,20624.638644,25.715222


In [24]:
df_cleaned['golden_cross'] = df_cleaned.groupby('ticker').apply(
    lambda x: ((x['ema_50'] > x['ema_200']) & (x['ema_50'].shift(1) <= x['ema_200'].shift(1))).astype(int)
).reset_index(level=0, drop=True)

df_cleaned['death_cross'] = df_cleaned.groupby('ticker').apply(
    lambda x: ((x['ema_50'] < x['ema_200']) & (x['ema_50'].shift(1) >= x['ema_200'].shift(1))).astype(int)
).reset_index(level=0, drop=True)

df_cleaned['return'] = df_cleaned.groupby('ticker')['close'].pct_change()

df_cleaned['macd_cross'] = df_cleaned.groupby('ticker').apply(
    lambda x: ((x['macd'] > x['macd_signal']) & (x['macd'].shift(1) <= x['macd_signal'].shift(1))).astype(int) -
              ((x['macd'] < x['macd_signal']) & (x['macd'].shift(1) >= x['macd_signal'].shift(1))).astype(int)
).reset_index(level=0, drop=True)

df_cleaned['bollinger_pct'] = (df_cleaned['close'] - df_cleaned['bollinger_lband']) / (df_cleaned['bollinger_hband'] - df_cleaned['bollinger_lband'])

df_cleaned['bollinger_bw'] = (df_cleaned['bollinger_hband'] - df_cleaned['bollinger_lband']) / ((df_cleaned['bollinger_hband'] + df_cleaned['bollinger_lband']) / 2)


df_cleaned.head()

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,...,rsi,bollinger_hband,bollinger_lband,mfi,golden_cross,death_cross,return,macd_cross,bollinger_pct,bollinger_bw
0,AAV,2022-03-24 00:00,22415.84,22512.46,21836.12,22125.98,236100,22915.445513,22915.445513,-298.539403,...,43.357123,24208.233901,21193.504099,20.534903,0,0,NaN,0,0.309307,0.132802
1,AAV,2022-03-25 00:00,22415.84,22512.46,21932.74,22029.36,285300,22880.697062,22880.697062,-303.332986,...,42.374002,24147.442352,21128.689648,22.420285,0,0,-0.004367,0,0.298358,0.133349
2,AAV,2022-03-28 00:00,22029.36,22029.36,21449.64,21642.88,426300,22832.155216,22832.155216,-334.462170,...,38.603353,24063.272540,21029.281460,23.695297,0,0,-0.017544,0,0.202241,0.134567
3,AAV,2022-03-30 00:00,21739.50,21739.50,20193.58,20869.92,478700,22755.204816,22755.204816,-416.700223,...,32.394605,23930.189747,20853.180253,24.491967,0,0,-0.035714,0,0.005440,0.137418
4,AAV,2022-04-01 00:00,20676.68,20676.68,20193.58,20580.06,380000,22669.905019,22669.905019,-499.505761,...,30.418830,23849.547356,20624.638644,25.715222,0,0,-0.013889,0,-0.013823,0.145024


In [25]:
df_cleaned.dropna(inplace=True)
df_cleaned = df_cleaned.reset_index()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63651 entries, 0 to 63650
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            63651 non-null  int64  
 1   ticker           63651 non-null  object 
 2   timestamp        63651 non-null  object 
 3   open             63651 non-null  float64
 4   high             63651 non-null  float64
 5   low              63651 non-null  float64
 6   close            63651 non-null  float64
 7   volume           63651 non-null  int64  
 8   ema_50           63651 non-null  float64
 9   ema_200          63651 non-null  float64
 10  macd             63651 non-null  float64
 11  macd_signal      63651 non-null  float64
 12  macd_diff        63651 non-null  float64
 13  rsi              63651 non-null  float64
 14  bollinger_hband  63651 non-null  float64
 15  bollinger_lband  63651 non-null  float64
 16  mfi              63651 non-null  object 
 17  golden_cross

In [26]:
df_cleaned.columns

Index(['index', 'ticker', 'timestamp', 'open', 'high', 'low', 'close',
       'volume', 'ema_50', 'ema_200', 'macd', 'macd_signal', 'macd_diff',
       'rsi', 'bollinger_hband', 'bollinger_lband', 'mfi', 'golden_cross',
       'death_cross', 'return', 'macd_cross', 'bollinger_pct', 'bollinger_bw'],
      dtype='object')

In [27]:
df_cleaned = df_cleaned.drop("index", axis =1)

In [28]:
df_cleaned.head()

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,...,rsi,bollinger_hband,bollinger_lband,mfi,golden_cross,death_cross,return,macd_cross,bollinger_pct,bollinger_bw
0,AAV,2022-03-25 00:00,22415.84,22512.46,21932.74,22029.36,285300,22880.697062,22880.697062,-303.332986,...,42.374002,24147.442352,21128.689648,22.420285,0,0,-0.004367,0,0.298358,0.133349
1,AAV,2022-03-28 00:00,22029.36,22029.36,21449.64,21642.88,426300,22832.155216,22832.155216,-334.462170,...,38.603353,24063.272540,21029.281460,23.695297,0,0,-0.017544,0,0.202241,0.134567
2,AAV,2022-03-30 00:00,21739.50,21739.50,20193.58,20869.92,478700,22755.204816,22755.204816,-416.700223,...,32.394605,23930.189747,20853.180253,24.491967,0,0,-0.035714,0,0.005440,0.137418
3,AAV,2022-04-01 00:00,20676.68,20676.68,20193.58,20580.06,380000,22669.905019,22669.905019,-499.505761,...,30.418830,23849.547356,20624.638644,25.715222,0,0,-0.013889,0,-0.013823,0.145024
4,AAV,2022-04-04 00:00,20580.06,22319.22,20580.06,21739.50,386400,22633.418548,22633.418548,-466.198582,...,44.896235,23670.327195,20629.942805,36.741016,0,0,0.056338,0,0.364940,0.137263


In [29]:
df_cleaned.to_excel(r"cleaned data\HNXINDEX_cleaned.xlsx")